In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

import plotly.graph_objects as go

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data load

In [2]:
#try to parse the dates right at the beginning 
# it works out of the box if the date was stored ISO YYYY-MM-DD format

df_analyse = pd.read_csv('../data/processed/COVID_small_flat_table.csv', sep=';',
parse_dates = [0])
df_analyse.sort_values('date', ascending = True).tail()

,date,Italy,US,Spain,Germany,"Korea, South"
875,2022-06-15,17736696,85942237,12515127,27096571,18256457
876,2022-06-16,17773764,86058228,12515127,27124689,18263643
877,2022-06-17,17809934,86216418,12563399,27204953,18270481
878,2022-06-18,17844905,86230982,12563399,27204955,18276552
879,2022-06-19,17879160,86246101,12563399,27211896,18280090


# Helper functions

In [3]:
def quick_plot(x_in, df_input, y_scale = 'log', slider = False):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()

    for each in df_input.columns:
        fig.add_trace(go.Scatter(
                        x=x_in,
                        y=df_input[each],
                        name=each,
                        opacity=0.8))
    
    fig.update_layout(autosize=True,
        width=1024,
        height=768,
        font=dict(
            family="PT Sans, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y_scale),
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()



In [4]:
quick_plot(df_analyse.date,
           df_analyse.iloc[:,1:],
           y_scale='log',
           slider=True)

In [5]:
threshold = 100

In [6]:
compare_list = []
for pos, country in enumerate(df_analyse.columns[1:]):
    compare_list.append(np.array(df_analyse[country][df_analyse[country] > threshold]))

In [7]:
pd_sync_timelines = pd.DataFrame(compare_list, index = df_analyse.columns[1:]).T

In [8]:
pd_sync_timelines['date'] = np.arange(pd_sync_timelines.shape[0])

In [9]:
pd_sync_timelines.head()

,Italy,US,Spain,Germany,"Korea, South",date
0,155.0,107.0,120.0,117.0,104.0,0
1,229.0,184.0,165.0,150.0,204.0,1
2,322.0,237.0,222.0,188.0,433.0,2
3,453.0,403.0,259.0,240.0,602.0,3
4,655.0,519.0,400.0,349.0,833.0,4


In [10]:
quick_plot(pd_sync_timelines.date,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True)

# Doubling Rate 
$N(t) = N_0*2^t/T$

In [11]:
def doubling_rate(N_0, t, T_d):
    return N_0*np.power(2, t/T_d)

In [12]:
max_days = 14

norm_slopes = {
    #'doubling every day': doubling_rate(100, np.arange(10), 1),
    'doubling every 2 days': doubling_rate(100, np.arange(20), 2), 
    'doubling every 4 days': doubling_rate(100, np.arange(20), 4), 
    'doubling every 10 days': doubling_rate(100, np.arange(20), 10), 

}

In [13]:
pd_sync_timelines_w_slope = pd.concat([pd.DataFrame(norm_slopes), pd_sync_timelines], axis = 1)

In [14]:
quick_plot(pd_sync_timelines_w_slope.date,
           pd_sync_timelines_w_slope.iloc[:,0:5],
           y_scale='log',
           slider=True)

In [15]:
pd_sync_timelines_w_slope.to_csv('../data/processed/COVID_small_sync_timeline_table.csv',sep=';',index=False)

# Understanding Linear Regression

In [16]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept= False)

In [17]:
l_vec = len(df_analyse['Germany'])
X = np.arange(l_vec-5).reshape(-1, 1)
y = np.log(np.array(df_analyse['Germany'][5:]))

In [18]:
reg.fit(X, y)

LinearRegression(fit_intercept=False)

In [19]:
X_hat = np.arange(l_vec).reshape(-1, 1)
Y_hat = reg.predict(X_hat)

In [20]:
LR_inspect = df_analyse[['date', 'Germany']].copy()

In [21]:
LR_inspect['prediction'] = np.exp(Y_hat)

In [22]:
quick_plot(LR_inspect.date,
           LR_inspect.iloc[:,1:],
           y_scale='log',
           slider=True)

# Doubling Rate - Piecewise Linear Regression

In [23]:
reg = linear_model.LinearRegression(fit_intercept= True)
l_vec = len(df_analyse['Germany'])
X = np.arange(l_vec-50).reshape(-1, 1)
y = np.array(df_analyse['Germany'][50:])

In [24]:
from scipy import signal

In [25]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  
country_list=df_analyse.columns[1:]

In [26]:
for each in country_list:
    df_analyse[each+'_filter'] = signal.savgol_filter(df_analyse[each], 
19, # window size used for filtering 
3) # order of fitted polynomial

In [27]:
filter_cols = ['Italy_filter', 'US_filter', 'Spain_filter', 'Germany_filter', 'Korea, South_filter']

In [28]:
start_pos = 5
quick_plot(df_analyse.date[start_pos:],
           df_analyse[filter_cols].iloc[start_pos:,:],
           y_scale='log',
           slider = True)

In [29]:
def get_rate_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

In [30]:
def doubling_time(in_array):
    ''' Use a classical doubling time formular, 
     see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [31]:
days_back = 3
for pos,country in enumerate(country_list):
    df_analyse[country+'_DR'] = df_analyse[country].rolling(window = days_back, min_periods = days_back).apply(get_rate_via_regression, raw = False)

In [32]:
days_back = 3
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR'] = df_analyse[country].rolling(window = days_back, min_periods = days_back).apply(get_rate_via_regression, raw = False)

In [33]:
df_analyse['Germany_DR_math'] = df_analyse['Germany'].rolling(
                                window = days_back,
                                min_periods = days_back).apply(doubling_time, raw = False)

In [34]:
days_back = 3
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR'] = df_analyse[country].rolling(window = days_back, min_periods = days_back).apply(get_rate_via_regression, raw = False)

In [35]:
df_analyse.columns

Index(['date', 'Italy', 'US', 'Spain', 'Germany', 'Korea, South',
       'Italy_filter', 'US_filter', 'Spain_filter', 'Germany_filter',
       'Korea, South_filter', 'Italy_DR', 'US_DR', 'Spain_DR', 'Germany_DR',
       'Korea, South_DR', 'Italy_filter_DR', 'US_filter_DR', 'Spain_filter_DR',
       'Germany_filter_DR', 'Korea, South_filter_DR', 'Germany_DR_math'],
      dtype='object')

In [36]:
start_pos = 40
quick_plot(df_analyse.date,
           df_analyse.iloc[start_pos:,[11, 12, 13, 14]],
           y_scale='linear',
           slider = True)

In [37]:
start_pos = 40
quick_plot(df_analyse.date,
           df_analyse.iloc[start_pos:,[16, 17, 18, 19]],
           y_scale='linear',
           slider = True)